In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import src.pytorch_utils as ptu
import src.dataset as dset
from src import bilstm

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models_path = 'models'

In [2]:
train_dataset = dset.DataSet('data/train.labeled', tqdm_bar=True, pad=False)
# test_dataset = dset.DataSet('data/test.labeled', tqdm_bar=True)
test_dataset = dset.DataSet('data/test.labeled', train_dataset=train_dataset, tqdm_bar=True, pad=False)
# comp_dataset = dset.DataSet('data/comp.unlabeled', train_dataset=train_dataset, tagged=False, tqdm_bar=True)

100%|██████████| 25325/25325 [00:02<00:00, 8795.43it/s]


In [3]:
# train_dataset_torch = torch.utils.data.TensorDataset(train_dataset.X, train_dataset.y)
# test_dataset_torch = torch.utils.data.TensorDataset(test_dataset.X, test_dataset.y)

train_dataset_torch = list(zip(train_dataset.X, train_dataset.y))
test_dataset_torch = list(zip(test_dataset.X, test_dataset.y))

In [4]:
class BiLSTM(nn.Module):
    def __init__(self, train_dataset, word_embed_dim, tag_embed_dim, hidden_dim, num_layers, bias, p_dropout):
        super(BiLSTM, self).__init__()

        self.word_embedding_layer = nn.Embedding(num_embeddings=len(train_dataset.words),
                                                 embedding_dim=word_embed_dim,
                                                 padding_idx=int(train_dataset.special[dset.PAD]))

        self.tag_embedding_layer = nn.Embedding(num_embeddings=len(train_dataset.tags),
                                                embedding_dim=tag_embed_dim,
                                                padding_idx=int(train_dataset.special[dset.PAD]))

        self.lstm = nn.LSTM(input_size=word_embed_dim + tag_embed_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            bias=bias,
                            dropout=p_dropout,
                            batch_first=False,
                            bidirectional=True)

        self.linear = nn.Linear(int(hidden_dim*2),
                                len(train_dataset.token_heads),
                                bias=bias)

    def forward(self, x, prints=False):
        if prints:
            print('input', x.shape)
        word_embeds = self.word_embedding_layer(x[:, :, 0]) # [batch_size, max_sentence_len, word_embed_dim]
        if prints:
            print('word_embeds', word_embeds.shape)
        tag_embeds = self.tag_embedding_layer(x[:, :, 1])   # [batch_size, max_sentence_len, tag_embed_dim]
        if prints:
            print('tag_embeds', tag_embeds.shape)
        x = torch.cat((word_embeds, tag_embeds), -1)         # []
        if prints:
            print('cat', x.shape)
        x, _ = self.lstm(x.view(x.shape[1], x.shape[0], -1))    # [seq_length, batch_size, 2*hidden_dim]
        if prints:
            print('lstm', x.shape)
        x = self.linear(x.transpose(1, 0))
        if prints:
            print('linear', x.shape)
        x = F.log_softmax(x, dim=2)
        if prints:
            print('log_softmax', x.shape)
        x = x.transpose(2, 1)
        if prints:
            print('transpose', x.shape)
        return x

In [5]:
version = 0.0

model = BiLSTM(train_dataset, word_embed_dim=300, tag_embed_dim=300, hidden_dim=512, num_layers=1, bias=True, p_dropout=0.0)

checkpoint = ptu.Checkpoint(models_path=models_path,
                            version=version,
                            model=model,
                            score=lambda y_true, y_pred: (np.array(y_true) == np.array(y_pred)).mean(),
                            decision_func=lambda x: x.argmax(axis=1),#.view(x.shape[0], -1),
                            seed=42,
                            optimizer=torch.optim.Adam,
                            criterion=nn.NLLLoss,
                            save=False,
                            prints=False,
                           )

In [11]:
hyperparam_list = [
    {'train_epochs': 50, 'batch_size': 32, 'optimizer_params': {'lr': 4e-4, 'weight_decay': 0}, 'lr_decay': 0.0},
#     {'train_epochs': 10, 'batch_size': 32, 'optimizer_params': {'lr': 1e-4, 'weight_decay': 0}, 'p_dropout': 0.5, 'lr_decay': 0.07},
#     {'train_epochs': 10, 'batch_size': 64, 'optimizer_params': {'lr': 1e-5, 'weight_decay': 0}, 'p_dropout': 0.5, 'lr_decay': 0.07},
#     {'train_epochs': 10, 'batch_size': 128, 'optimizer_params': {'lr': 1e-6, 'weight_decay': 0}, 'p_dropout': 0.5, 'lr_decay': 0.07},
]

for session in hyperparam_list:
    checkpoint.train(device=device,
                     train_dataset=train_dataset_torch,
                     val_dataset=test_dataset_torch,
                     prints=True,
                     epochs_save=1,
                     save=False,
                     **session)

KeyboardInterrupt: 

In [ ]:
model = model.to(device)
model.train()

loss_sum = np.array([])
y_pred = np.array([])
y_true = np.array([])

counter = 0
for batch in torch.utils.data.DataLoader(dataset=test_dataset_torch, batch_size=1, shuffle=True):
    for i, _ in enumerate(batch):
        batch[i] = batch[i].to(device)
    out = checkpoint.model.forward(*batch[:-1], prints=True)
    print('out', out.shape)
    print('true', batch[-1].long().shape)
    loss = checkpoint.criterion(out, batch[-1].long())

    loss_sum = np.append(loss_sum, float(loss.data))
    
    y_pred = np.append(y_pred, checkpoint.decision_func(out.detach().cpu().numpy()))
    
    y_true = np.append(y_true, batch[-1].detach().cpu().numpy())
    counter += 1
    if counter > 5:
        break

In [ ]:
#### matan's code below

In [ ]:
## dataset class, need to review and keep only the relevant
class Dependency_Parser_Dataset(nn.Module):
    def __init__(self, word_dict, pos_dict, dir_path: str, subset: str, 
                 padding=False, word_embeddings=None):
        super().__init__()
        self.subset = subset # One of the following: [train, test]
        self.file = dir_path + subset + ".wtag"
        self.datareader = PosDataReader(self.file, word_dict, pos_dict)
        self.vocab_size = len(self.datareader.word_dict)
        if word_embeddings:
            self.word_idx_mappings, self.idx_word_mappings, self.word_vectors = word_embeddings
        else:
            self.word_idx_mappings, self.idx_word_mappings, self.word_vectors = ## need to add our word emb hereself.init_word_embeddings(self.datareader.word_dict)
        self.pos_idx_mappings, self.idx_pos_mappings = ## need to add our word emb here self.init_pos_vocab(self.datareader.pos_dict)
        
        self.pad_idx = self.word_idx_mappings.get(PAD_TOKEN)
        self.unknown_idx = self.word_idx_mappings.get(UNKNOWN_TOKEN)
        self.word_vector_dim = self.word_vectors.size(-1)
        self.sentence_lens = [len(sentence) for sentence in self.datareader.sentences]
        self.max_seq_len = max(self.sentence_lens)
        self.sentences_dataset = self.convert_sentences_to_dataset(padding)
    
    def get_word_embeddings(self):
        return self.word_idx_mappings, self.idx_word_mappings, self.word_vectors

    
    def init_pos_vocab(self, pos_dict):
        idx_pos_mappings = sorted([self.word_idx_mappings.get(token) for token in SPECIAL_TOKENS])
        pos_idx_mappings = {self.idx_word_mappings[idx]: idx for idx in idx_pos_mappings}
        
        
    def convert_sentences_to_dataset(self, padding):
        sentence_word_idx_list = list()
        sentence_pos_idx_list = list()
        sentence_len_list = list()
        for sentence_idx, sentence in enumerate(self.datareader.sentences):
            words_idx_list = []
            pos_idx_list = []
            for word, pos in sentence:
                words_idx_list.append(self.word_idx_mappings.get(word))
                pos_idx_list.append(self.pos_idx_mappings.get(pos))
            sentence_len = len(words_idx_list)
            # if padding:
            #     while len(words_idx_list) < self.max_seq_len:
            #         words_idx_list.append(self.word_idx_mappings.get(PAD_TOKEN))
            #         pos_idx_list.append(self.pos_idx_mappings.get(PAD_TOKEN))
            sentence_word_idx_list.append(torch.tensor(words_idx_list, dtype=torch.long, requires_grad=False))
            sentence_pos_idx_list.append(torch.tensor(pos_idx_list, dtype=torch.long, requires_grad=False))
            sentence_len_list.append(sentence_len)
        
        # if padding:
        #     all_sentence_word_idx = torch.tensor(sentence_word_idx_list, dtype=torch.long)
        #     all_sentence_pos_idx = torch.tensor(sentence_pos_idx_list, dtype=torch.long)
        #     all_sentence_len = torch.tensor(sentence_len_list, dtype=torch.long, requires_grad=False)
        #     return TensorDataset(all_sentence_word_idx, all_sentence_pos_idx, all_sentence_len)
            
        return {i: sample_tuple for i, sample_tuple in enumerate(zip(sentence_word_idx_list,
                                                                     sentence_pos_idx_list,
                                                                     sentence_len_list))}

In [ ]:
## our model
class Dnn_Dependency_Parser(nn.Module):
    def __init__(self, word_embeddings, hidden_dim, word_vocab_size, tag_vocab_size):
        super(Dnn_Dependency_Parser, self).__init__()
        emb_dim = word_embeddings.shape[1]
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.word_embedding = ## need to add our embeddin - nn.Embedding(word_vocab_size, word_embedding_dim)
        # self.word_embedding = nn.Embedding.from_pretrained(word_embeddings, freeze=False)
        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=hidden_dim, num_layers=2, bidirectional=True, batch_first=False)
        self.hidden2tag = nn.Linear(hidden_dim*2, tag_vocab_size)

        
    def forward(self, word_idx_tensor):
        embeds = self.word_embedding(word_idx_tensor.to(self.device))   # [batch_size, seq_length, emb_dim]      
        lstm_out, _ = self.lstm(embeds.view(embeds.shape[1], 1, -1))    # [seq_length, batch_size, 2*hidden_dim]
        tag_space = self.hidden2tag(lstm_out.view(embeds.shape[1], -1)) # [seq_length, tag_dim]
        tag_scores = F.log_softmax(tag_space, dim=1)                    # [seq_length, tag_dim]
        return tag_scores


In [ ]:
#CUDA_LAUNCH_BLOCKING=1  

EPOCHS = 15
WORD_EMBEDDING_DIM = 100 ## need to decide if this is the right DIM
HIDDEN_DIM = 1000 ## need to decide if this is the right DIM
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)

##need to decide whihc parameters are relevant
model = Dnn_Dependency_Parser(train_dataloader.dataset.word_vectors, HIDDEN_DIM, word_vocab_size, tag_vocab_size)

if device == "cuda":
    model.cuda()

# Define the loss function as the Negative Log Likelihood loss (NLLLoss)

## no need of that, i implement Negative_log_Likelihood_Loss
#loss_function = nn.NLLLoss()

# We will be using a simple SGD optimizer to minimize the loss function
## we are ok with Adam? need to change?
optimizer = optim.Adam(model.parameters(), lr=0.01)
acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1

# Training start
print("Training Started")
accuracy_list = []
loss_list = []
epochs = EPOCHS
for epoch in range(epochs):
    acc = 0 # to keep track of accuracy
    printable_loss = 0 # To keep track of the loss value
    i = 0
    for batch_idx, input_data in enumerate(train_dataloader):
        i += 1
        words_idx_tensor, pos_idx_tensor, sentence_length = input_data
        
        sentence_scores = model(words_idx_tensor)
        sentence_scores = ## need to fix it ----tag_scores.unsqueeze(0).permute(0,2,1)
        #print("tag_scores shape -", tag_scores.shape)
        #print("pos_idx_tensor shape -", pos_idx_tensor.shape)
        loss = Negative_log_Likelihood_Loss(dataset, network_parameters) # need to fix network_parameters
        loss = loss / acumulate_grad_steps
        loss.backward()

        if i % acumulate_grad_steps == 0:
            optimizer.step()
            model.zero_grad()
        printable_loss += loss.item()## we need to change loss.item() to our lass, i think it will be only loss
        _, indices = torch.max(sentence_scores, 1)
        # print("tag_scores shape-", tag_scores.shape)
        # print("indices shape-", indices.shape)
        # acc += indices.eq(pos_idx_tensor.view_as(indices)).mean().item()
        acc += torch.mean(torch.tensor(pos_idx_tensor.to("cpu") == indices.to("cpu"), dtype=torch.float))##i think we should fix it
    printable_loss = printable_loss / len(train)
    acc = acc / len(train)
    loss_list.append(float(printable_loss))
    accuracy_list.append(float(acc))
    test_acc = evaluate()
    e_interval = i
    print("Epoch {} Completed,\tLoss {}\tAccuracy: {}\t Test Accuracy: {}".format(epoch + 1, np.mean(loss_list[-e_interval:]), np.mean(accuracy_list[-e_interval:]), test_acc))
  

In [ ]:
## loss function
def Negative_log_Likelihood_Loss(dataset, network_parameters):
    loss = 0
    for x_i, y_i in dataset:
        softmax_score = softmax(y_i)
        for head, modifier in y_i:
            loss -=(1/absoulte_y_i(y_i))*mat.log(softmax_score(head,modifer))
              
def absoulte_y_i(y_i):
    return len(y_i[:0])

In [ ]:
## evaluate functino- i used the function they share with us,looks ok for me 
def evaluate(test_dataloader):
    acc = 0
    with torch.no_grad():
        for batch_idx, input_data in enumerate(test_dataloader):
            
            words_idx_tensor, pos_idx_tensor, sentence_length = input_data  
            tag_scores = model(words_idx_tensor)
            tag_scores = tag_scores.unsqueeze(0).permute(0,2,1)
            
            _, indices = torch.max(tag_scores, 1)
            acc += torch.mean(torch.tensor(pos_idx_tensor.to("cpu") == indices.to("cpu"), dtype=torch.float))
        acc = acc / len(test)
    return acc